<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/add_top_type/colabs/Dane_z_dodatkowymi_kolumnami.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!git clone https://github.com/PiotrMaciejKowalski/BigData2022-actors.git
!mv /content/BigData2022-actors/* .
!mv /content/BigData2022-actors/.* .
!rmdir /content/BigData2022-actors/

Cloning into 'BigData2022-actors'...
remote: Enumerating objects: 1594, done.
remote: Counting objects: 100% (287/287), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 1594 (delta 199), reused 165 (delta 110), pack-reused 1307
Receiving objects: 100% (1594/1594), 6.01 MiB | 22.47 MiB/s, done.
Resolving deltas: 100% (968/968), done.
mv: cannot move '/content/BigData2022-actors/.' to './.': Device or resource busy
mv: cannot move '/content/BigData2022-actors/..' to './..': Device or resource busy


In [3]:
!git checkout add_top_type

Branch 'add_top_type' set up to track remote branch 'add_top_type' from 'origin'.
Switched to a new branch 'add_top_type'


In [4]:
!chmod +x setup_sparka.sh
!./setup_sparka.sh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 18.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=29566eb37fe22e7884ee387480f8e1ea9ee661c8dc7c2d64740341d49984fe3f
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
--2023-01-15 22:14:07--  https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274099817 

In [5]:
import pandas as pd
from lib.pyspark_init import create_spark_context
from lib.adding_columns import add_all_columns
from lib.const import DATA_WITH_AWARDS, TRAIN_DATA, VALIDATE_DATA, TEST_DATA

In [6]:
spark = create_spark_context()
spark

In [7]:
!chmod +x download_data.sh
!./download_data.sh

--2023-01-15 22:14:24--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 52.84.125.36, 52.84.125.109, 52.84.125.29, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|52.84.125.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 239457320 (228M) [binary/octet-stream]
Saving to: ‘name.basics.tsv.gz’

name.basics.tsv.gz  100%[===================>] 228.36M  85.6MB/s    in 2.7s    

2023-01-15 22:14:26 (85.6 MB/s) - ‘name.basics.tsv.gz’ saved [239457320/239457320]

--2023-01-15 22:14:26--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 52.84.125.36, 52.84.125.109, 52.84.125.29, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|52.84.125.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 293608623 (280M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz’

title.akas.tsv.gz   100%[===================>] 280.01M  88.

In [8]:
%%time
data = spark.read.parquet(DATA_WITH_AWARDS)
data.show()

CPU times: user 50.8 ms, sys: 3.9 ms, total: 54.7 ms
Wall time: 8.46 s


In [9]:
%%time
data = add_all_columns(spark, data)
data.show()

+---------+--------------------+--------------------+--------------------+--------------------+---------+-------+--------------------+--------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+-------------+------------+------------+-------------+--------+---------------------+---------+---------------------+---------+-------------------+-------+--------+--------+--------------------+
|   nconst|              tconst|           titleType|       originalTitle|             isAdult|startYear|endYear|              genres|category|          characters|    primaryName|      knownForTitles|         year_oscars|     category_oscars|         film_oscars|       winner_oscars|         year_globes|     category_globes|         film_globes|          win_globes|year_emmy|category_emmy|nominee_emmy|company_emmy|produce

In [10]:
%%time
train_data, validate_data, test_data = data.randomSplit([0.7, 0.1, 0.2], seed=1)

CPU times: user 8.02 ms, sys: 1.84 ms, total: 9.86 ms
Wall time: 636 ms


In [11]:
%%time
data.write.parquet(TRAIN_DATA)

AnalysisException: ignored

In [12]:
%%time
data.write.parquet(VALIDATE_DATA)

AnalysisException: ignored

In [13]:
%%time
data.write.parquet(TEST_DATA)

AnalysisException: ignored